In [ ]:
# %%
# # 1. Exploration du jeu de données de nanoparticules
#
# Ce notebook a pour but de charger, visualiser et comprendre notre jeu de données synthétiques.
#
# Étapes :
# 1. Importer les bibliothèques nécessaires.
# 2. Charger les jeux de données d'entraînement et de validation en utilisant notre classe `NanoparticleDataset`.
# 3. Visualiser quelques échantillons (image + masque) pour vérifier que les données sont correctement chargées.
# 4. Appliquer et visualiser les transformations d'augmentation de données.

# %%
# Importation des bibliothèques
import torch
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
import sys
import os

# Ajout du répertoire racine au path pour pouvoir importer les modules du projet
# Cela permet d'exécuter ce notebook depuis le dossier 'notebooks/'
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from utils.dataset import NanoparticleDataset
from utils.transforms import get_train_transforms, get_val_transforms

# %%
# ## 2. Chargement des Données
#
# Nous allons maintenant créer des instances de notre `NanoparticleDataset` pour les données d'entraînement et de validation. Nous n'appliquons aucune transformation pour le moment afin de voir les données brutes.

# %%
# Définition des chemins
TRAIN_IMG_DIR = "../data/train/images/"
TRAIN_MASK_DIR = "../data/train/masks/"
VAL_IMG_DIR = "../data/val/images/"
VAL_MASK_DIR = "../data/val/masks/"

# Création des datasets
# Note: On n'utilise pas de 'transform' ici pour voir les images originales
train_dataset_raw = NanoparticleDataset(image_dir=TRAIN_IMG_DIR, mask_dir=TRAIN_MASK_DIR)
val_dataset_raw = NanoparticleDataset(image_dir=VAL_IMG_DIR, mask_dir=VAL_MASK_DIR)

print(f"Nombre d'images d'entraînement : {len(train_dataset_raw)}")
print(f"Nombre d'images de validation : {len(val_dataset_raw)}")

# %%
# ## 3. Visualisation des Échantillons
#
# Affichons quelques paires image/masque pour nous assurer que tout est correct. L'image de gauche doit être la version bruitée, et celle de droite le masque de segmentation parfait.

# %%
def visualize_sample(dataset, num_samples=5):
    fig, axes = plt.subplots(num_samples, 2, figsize=(8, num_samples * 4))
    fig.suptitle("Échantillons du jeu de données (Image vs Masque)", fontsize=16)

    for i in range(num_samples):
        # Récupérer un échantillon aléatoire
        idx = np.random.randint(0, len(dataset))
        image, mask = dataset[idx]

        # Afficher l'image
        axes[i, 0].imshow(image, cmap='gray')
        axes[i, 0].set_title(f"Image {idx}")
        axes[i, 0].axis('off')

        # Afficher le masque
        axes[i, 1].imshow(mask, cmap='gray')
        axes[i, 1].set_title(f"Masque {idx}")
        axes[i, 1].axis('off')

    plt.tight_layout()
    plt.show()

# Visualiser des échantillons du jeu d'entraînement
visualize_sample(train_dataset_raw)

# %%
# ## 4. Test des Transformations d'Augmentation
#
# L'augmentation de données est cruciale pour que le modèle apprenne à généraliser. Nous allons maintenant appliquer les transformations définies dans `utils/transforms.py` et visualiser leur effet sur une image et son masque.
#
# On s'attend à voir des rotations, des retournements, etc., appliqués de manière identique à l'image et à son masque.

# %%
# Récupérer les pipelines de transformation
IMAGE_HEIGHT = 256
IMAGE_WIDTH = 256
train_transforms = get_train_transforms(IMAGE_HEIGHT, IMAGE_WIDTH)

# Créer un nouveau dataset avec les transformations
train_dataset_aug = NanoparticleDataset(
    image_dir=TRAIN_IMG_DIR,
    mask_dir=TRAIN_MASK_DIR,
    transform=train_transforms
)

# %%
def visualize_augmented_sample(dataset, num_samples=5):
    fig, axes = plt.subplots(num_samples, 2, figsize=(8, num_samples * 4))
    fig.suptitle("Échantillons Augmentés (Image vs Masque)", fontsize=16)

    for i in range(num_samples):
        idx = np.random.randint(0, len(dataset))
        image_tensor, mask_tensor = dataset[idx]

        # Convertir les tenseurs en arrays numpy pour l'affichage
        # La transformation ToTensorV2 place la dimension du canal en premier (C, H, W)
        # Nous devons la remettre à la fin pour matplotlib (H, W, C)
        image = image_tensor.permute(1, 2, 0).numpy()
        mask = mask_tensor.permute(1, 2, 0).numpy()

        # Afficher l'image augmentée
        axes[i, 0].imshow(image, cmap='gray')
        axes[i, 0].set_title(f"Image Augmentée {idx}")
        axes[i, 0].axis('off')

        # Afficher le masque augmenté
        axes[i, 1].imshow(mask, cmap='gray')
        axes[i, 1].set_title(f"Masque Augmenté {idx}")
        axes[i, 1].axis('off')

    plt.tight_layout()
    plt.show()

# Visualiser des échantillons augmentés
visualize_augmented_sample(train_dataset_aug)

# %%
# ### Conclusion de l'exploration
#
# - Le chargement des données fonctionne comme prévu.
# - Les paires image/masque correspondent.
# - Les transformations d'augmentation sont appliquées correctement à la fois aux images et aux masques, ce qui est essentiel pour un entraînement cohérent.
#
# Le projet est maintenant prêt pour la phase d'entraînement.